<a href="https://colab.research.google.com/github/lunayee/bikesharing/blob/main/bikesharing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 載入套件

In [2]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
%matplotlib inline

# 上傳資料

In [3]:
train = pd.read_csv('D:/Python/bikesharing/train.csv')
test = pd.read_csv('D:/Python/bikesharing/test.csv')
submit = pd.read_csv('D:/Python/bikesharing/sampleSubmission.csv')

FileNotFoundError: ignored

# 特徵工程

## (一)刪除oulier

In [ ]:
#刪除oulier 有問題
print ('shape of the before outlier:',train.shape)
train = train[np.abs(train['count']-train['count'].mean())<=train['count'].std()*3]
print('shape of the after outlier:',train.shape)
train.describe()

## (二)合併

In [ ]:
#先把test&train和在一起最後再分開，才不用相同動作一直做
data = train.append(test)
data.reset_index(inplace=True)
data.drop('index',axis=1)

## (三)時間格式

In [ ]:
#model沒辦法吃時間的格式，所以資料要做點處理
data['date'] = data['datetime'].apply(lambda x : x.split()[0])
data['hour'] = data['datetime'].apply(lambda x : x.split()[1].split(':')[0]).astype(int)
data['weekday']= data['date'].apply(lambda dataString : datetime.datetime.strptime(dataString,'%Y-%m-%d').weekday())
data['year'] = data['datetime'].apply(lambda x : x.split()[0].split('-')[0]).astype(int)
data['month'] = data['datetime'].apply(lambda x : x.split()[0].split('-')[1]).astype(int)

## (四)觀察資料

In [ ]:
fig,axes = plt.subplots(nrows=2,ncols=2)
fig.set_size_inches(24,10)
sns.distplot(data["temp"],ax=axes[0][0])
sns.distplot(data["atemp"],ax=axes[0][1])
sns.distplot(data["humidity"],ax=axes[1][0])
sns.distplot(data["windspeed"],ax=axes[1][1])

axes[0][0].set(xlabel='temp',title='distribution of temp')
axes[0][1].set(xlabel='atemp',title='distribution of atemp')
axes[0][1].set(xlabel='humidity',title='distribution of humidity')
axes[0][1].set(xlabel='windspeed',title='distribution of windspeed')

In [ ]:
#風速那部分會對資料預測造成嚴重得誤判，因此要把0的部分想辦法補值
datawind0 = data[data['windspeed']==0]
datawindnot0 = data[data['windspeed']!=0]
rfmodel_wind = RandomForestRegressor(n_estimators=1000,random_state=42)
windcolumn = ['season','weather','humidity','temp','atemp','year','month']
rfmodel_wind.fit(datawindnot0[windcolumn],datawindnot0['windspeed'])

wind0value = rfmodel_wind.predict(X=datawind0[windcolumn])


In [ ]:
datawind0.loc[:,'windspeed'] = wind0value
data = datawindnot0.append(datawind0)
data.reset_index(inplace=True)
data.drop('index',axis=1,inplace=True)
data.drop('level_0',axis=1)
sns.distplot(data['windspeed'])

# 測試值訓練值

In [ ]:
dataTrain = data[pd.notnull(data['count'])].sort_values(by=['datetime'])
dataTest = data[~pd.notnull(data['count'])].sort_values(by=['datetime'])

In [ ]:
#最後需要提交的檔案有時間資訊所以先提出
look,axes = plt.subplots(nrows=2,ncols=1)
look.set_size_inches(24,10)
dataDatatime = dataTest['datetime']
ylebel = dataTrain['count']
ylebellog = np.log(ylebel)
sns.distplot(ylebel,ax=axes[0])#太左偏
sns.distplot(ylebellog,ax=axes[1])#希望符合常態分配

In [ ]:
#丟掉不要的欄位
datadrop = ['casual','count','datetime','date','registered']
dataTrain = dataTrain.drop(datadrop,axis=1)
dataTest = dataTest.drop(datadrop,axis=1)
dataTest

In [ ]:
#隨機森林模型建置
rfmodel =RandomForestRegressor(n_estimators=1000,random_state=42)
rfmodel.fit(dataTrain,ylebellog)
submit = rfmodel.predict(X=dataTest)
submit

# 檔案提交

In [ ]:
#下載成比賽需要的檔案格式
submission = pd.DataFrame({
    'datetime':dataDatatime,
    'count': [max(0,x) for x in np.exp(submit)]
})

submission.to_csv('bike_predictions_RF.CSV',index=None)